In [2]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [3]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

In [4]:
link = pd.read_csv('./ml-latest-small/links.csv')

In [5]:
rating = pd.read_csv('./ml-latest-small/ratings.csv')

In [6]:
tags = pd.read_csv('./ml-latest-small/tags.csv')

In [7]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [8]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [9]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [10]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [11]:
movie_rating = movies.merge(rating, on='movieId', how='outer')

In [12]:
movie_rating

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09


In [13]:
movie_rating.isna().sum()

movieId       0
title         0
genres        0
userId       18
rating       18
timestamp    18
dtype: int64

In [14]:
movie_rating.dropna(inplace=True)

In [15]:
movie_rating['userId'].nunique()

610

In [16]:
stats = movie_rating[['rating', 'timestamp']].describe()
stats

,rating,timestamp
count,100836.000000,1.008360e+05
mean,3.501557,1.205946e+09
std,1.042529,2.162610e+08
min,0.500000,8.281246e+08
25%,3.000000,1.019124e+09
50%,3.500000,1.186087e+09
75%,4.000000,1.435994e+09
max,5.000000,1.537799e+09


In [17]:
print(dt.datetime.fromtimestamp(stats.loc['min', 'timestamp']))
print(dt.datetime.fromtimestamp(stats.loc['max', 'timestamp']))

1996-03-29 10:36:55
2018-09-24 07:27:30


In [19]:
movie_rating['Title'] = movie_rating['title'].str.split('(', n=1, expand=True)[0]

movie_rating['year_released'] = movie_rating['title'].str.split('(', n=1, expand=True)[1]

In [20]:
movie_rating

,movieId,title,genres,userId,rating,timestamp,Title,year_released
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,Toy Story,1995)
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,Toy Story,1995)
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,Toy Story,1995)
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,Toy Story,1995)
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,Toy Story,1995)
...,...,...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09,Black Butler: Book of the Atlantic,2017)
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09,No Game No Life: Zero,2017)
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09,Flint,2017)
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09,Bungo Stray Dogs: Dead Apple,2018)


In [24]:
# Drop original column title
movie_rating.drop('title',axis=1,inplace=True)

In [25]:
# Format year_released column

movie_rating['year_released'] = movie_rating.year_released.str.replace(')', '')

In [26]:
movie_rating

,movieId,genres,userId,rating,timestamp,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,Toy Story,1995
1,1,Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,Toy Story,1995
2,1,Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,Toy Story,1995
3,1,Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,Toy Story,1995
4,1,Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,Toy Story,1995
...,...,...,...,...,...,...,...
100849,193581,Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09,Black Butler: Book of the Atlantic,2017
100850,193583,Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09,No Game No Life: Zero,2017
100851,193585,Drama,184.0,3.5,1.537110e+09,Flint,2017
100852,193587,Action|Animation,184.0,3.5,1.537110e+09,Bungo Stray Dogs: Dead Apple,2018


In [27]:
#converted timestamp to datetime
movie_rating['timestamp'] = pd.to_datetime(movie_rating['timestamp'], unit='s')

In [28]:
movie_rating

,movieId,genres,userId,rating,timestamp,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,2000-07-30 18:45:03,Toy Story,1995
1,1,Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,1996-11-08 06:36:02,Toy Story,1995
2,1,Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,2005-01-25 06:52:26,Toy Story,1995
3,1,Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,2017-11-13 12:59:30,Toy Story,1995
4,1,Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,2011-05-18 05:28:03,Toy Story,1995
...,...,...,...,...,...,...,...
100849,193581,Action|Animation|Comedy|Fantasy,184.0,4.0,2018-09-16 14:44:42,Black Butler: Book of the Atlantic,2017
100850,193583,Animation|Comedy|Fantasy,184.0,3.5,2018-09-16 14:52:25,No Game No Life: Zero,2017
100851,193585,Drama,184.0,3.5,2018-09-16 14:56:45,Flint,2017
100852,193587,Action|Animation,184.0,3.5,2018-09-16 15:00:21,Bungo Stray Dogs: Dead Apple,2018


In [29]:
movie_rating['genres']=[row.strip().lower().replace('|',',') for row in movie_rating['genres']]

In [32]:
movie_rating['genres']=movie_rating['genres'].to_list()

In [33]:
movie_rating

,movieId,genres,userId,rating,timestamp,Title,year_released
0,1,"adventure,animation,children,comedy,fantasy",1.0,4.0,2000-07-30 18:45:03,Toy Story,1995
1,1,"adventure,animation,children,comedy,fantasy",5.0,4.0,1996-11-08 06:36:02,Toy Story,1995
2,1,"adventure,animation,children,comedy,fantasy",7.0,4.5,2005-01-25 06:52:26,Toy Story,1995
3,1,"adventure,animation,children,comedy,fantasy",15.0,2.5,2017-11-13 12:59:30,Toy Story,1995
4,1,"adventure,animation,children,comedy,fantasy",17.0,4.5,2011-05-18 05:28:03,Toy Story,1995
...,...,...,...,...,...,...,...
100849,193581,"action,animation,comedy,fantasy",184.0,4.0,2018-09-16 14:44:42,Black Butler: Book of the Atlantic,2017
100850,193583,"animation,comedy,fantasy",184.0,3.5,2018-09-16 14:52:25,No Game No Life: Zero,2017
100851,193585,drama,184.0,3.5,2018-09-16 14:56:45,Flint,2017
100852,193587,"action,animation",184.0,3.5,2018-09-16 15:00:21,Bungo Stray Dogs: Dead Apple,2018


## Split

In [34]:
movie_rating.columns

Index(['movieId', 'movieIdmovie_', 'title', 'genres', 'userId',
       'movieIdrating_', 'rating', 'timestamp'],
      dtype='object')

In [50]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(movie_rating[['userId', 'movieId', 'rating']], reader)

In [51]:
data

In [52]:
train, test = surprise.model_selection.train_test_split(data, random_state=42)

In [53]:
train

In [54]:
model = KNNBasic().fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [55]:
model2 = SVD().fit(train)

In [56]:
model3 = NMF().fit(train)

In [74]:
model4 = knns.KNNWithMeans().fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.


## Tuning

In [76]:
model = KNNBasic()

In [78]:
train.

In [77]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(model,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train)

AttributeError: 'Trainset' object has no attribute 'raw_ratings'

In [ ]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

## Cross Validation

In [66]:
CV1 = surprise.model_selection.split.KFold()

In [ ]:
surprise.model_selection.split.RepeatedKFold()